# Data Preprocessing

## Import the needed package

In [48]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, date
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

Build the folder path and load the data

In [ ]:
global_path=os.path.abspath('.')
df_train = pd.read_csv(global_path+'/data/train_users_2.csv')
df_test = pd.read_csv(global_path+'/data/test_users.csv')
df_sessions = pd.read_csv(global_path+'/data/sessions.csv')
df_age_gender_bkts = pd.read_csv(global_path+'/data/age_gender_bkts.csv')
df_countries = pd.read_csv(global_path+'/data/countries.csv')

## Preprocesse train_users_2 and test_users dataset

Get the target of the train user and delete the taraget from the data frame

In [3]:
label = df_train['country_destination']
df_train = df_train.drop(['country_destination'], axis=1)

Get the id of test which used in getting prediction value

In [4]:
id_test = df_test['id']

Conbine the train data and testing data

In [5]:
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all.index = df_all.id

Inputing this kind of missing value with -1 (missing values in train and test)<br>
'-unknown-' is another way of missing value, then = -1.

In [6]:
df_feature = df_all.fillna(-1)
df_feature = df_feature.replace('-unknown-', -1)

As shown in the analysis, the total missing value of data_first_booking is too high because this feature is not present at the tests users, and therefore, we won't need it at the modeling part

In [7]:
df_feature = df_feature.drop(['date_first_booking'], axis=1)
df_feature['number_null'] = np.array([sum(r == -1) for r in df_feature.values])

Clean the age data which is write in year.<br>
Set the people who is under 14 or is above 100 as the error number<br>
Return it back to df_feature dataframe

In [9]:
age = df_feature.age.values
age = [2015 - x if x >= 1920 else x for x in age]
age = [-1 if x<14 or x>100 else x for x in age]
df_feature['age'] = age

Seperate the age into different interval and map the user's age into the bucket<br>
Create the new feature 'age_bucket'

In [10]:
df_feature['age_bucket'] = pd.cut(age,
                           [np.nanmin(age),4,9,14,19,24,29,34,39,44,49,54,59,64,69,74,79,84,89,94,99,np.nanmax(age)],
                            labels = ["0-4", "5-9", "10-14", "15-19",
                                      "20-24", "25-29", "30-34", "35-39",
                                      "40-44", "45-49", "50-54", "55-59",
                                      "60-64", "65-69", "70-74", "75-79",
                                      "80-84", "85-89", "90-94", "95-99", "100+"])

## Preprocessing the countries data

In [11]:
df_countries['destination_language'] = df_countries['destination_language ']
df_countries = df_countries.drop(['destination_language '], axis=1)

Add new feature: 'country_language'

In [ ]:
df_countries['country_language'] = np.array([df_countries.country_destination.values[i]+"_"+df_countries.destination_language.values[i] for i in range(len(df_countries.country_destination.values))])

Reconstruction the countries dataframe

In [12]:
df_countries_reshape = pd.DataFrame()
for i in set(df_countries.country_language.values):
    df_countries_index = pd.DataFrame()
    # i <- "AU_eng"
    #return the row of country_language equals 'Au_eng'
    #print(type(i))
    countries_lan = df_countries['country_language'] == i
    countries_tag  = df_countries[countries_lan] 
    #delete the country_language column
    x = countries_tag.country_destination.values[0]
    countries_tag  = countries_tag.drop(['country_language'], axis=1)
    #reshape the dataframe
    df_countries_index['destination_language'] = countries_tag.destination_language.values
    df_countries_index['lat_destination_'+x] = countries_tag.lat_destination.values
    df_countries_index['lng_destination_'+x] = countries_tag.lng_destination.values
    df_countries_index['distance_km_'+x] = countries_tag.distance_km.values
    df_countries_index['destination_km2_'+x] = countries_tag.destination_km2.values
    df_countries_index['language_levenshtein_distance_'+x] = countries_tag.language_levenshtein_distance.values
    df_countries_reshape = pd.concat([df_countries_reshape,df_countries_index])

Set the destination_language as the language type which helps to match the language in df_feature

In [ ]:
df_countries_reshape['destination_language'] = [x[:2] for x in df_countries_reshape.destination_language]
df_countries_reshape = df_countries_reshape.groupby(['destination_language']).agg(np.sum)

Inputing this kind of missing value with -1 (missing values in train and test)

In [ ]:
df_countries_reshape = df_countries_reshape.fillna(0)

Set the index as the column name

In [50]:
df_countries_reshape['destination_language'] = df_countries_reshape.index

Merge the df_feature and df_countries_reshape by the laguage

In [13]:
df_feature = pd.merge(df_feature, df_countries_reshape,how ='left',left_on='language', right_on='destination_language')

## Preprocessing the age gender buckets data

In [15]:
#working on age_gender_bucket
df_age_gender_bkts = pd.merge(df_age_gender_bkts, df_countries, left_on='country_destination', right_on='country_destination')

#select the sub feature
df_age_gender_bkts = df_age_gender_bkts[["age_bucket", "country_destination", "gender", "population_in_thousands", "year", "destination_language"]].copy()

#create new feature -> "ca_eng"
df_age_gender_bkts['country_language'] = np.array([df_age_gender_bkts.country_destination.values[i]+"_"+df_age_gender_bkts.destination_language.values[i] for i in range(len(df_age_gender_bkts.country_destination.values))])

Reconstruction the age gender buckets dataframe

In [16]:
df_age_gender_bkts_reshape = pd.DataFrame()
for i in set(df_age_gender_bkts.country_language.values):
    df_age_gender_bkts_index = pd.DataFrame()
    # i <- "AU_eng"
    #return the row of country_language equals 'Au_eng'
    #print(type(i))
    age_gender_bkts_lan = df_age_gender_bkts['country_language'] == i
    age_gender_bkts_countries_tag  = df_age_gender_bkts[age_gender_bkts_lan] 
    #delete the country_language column
    x = age_gender_bkts_countries_tag.country_destination.values[0]
    age_gender_bkts_countries_tag  = age_gender_bkts_countries_tag.drop(['country_language'], axis=1)
    #reshape the dataframe
    df_age_gender_bkts_index['destination_language'] = age_gender_bkts_countries_tag.destination_language
    df_age_gender_bkts_index['population_in_thousands_'+x] = age_gender_bkts_countries_tag.population_in_thousands
    df_age_gender_bkts_index['age_bucket'] = age_gender_bkts_countries_tag.age_bucket
    df_age_gender_bkts_index['gender'] = age_gender_bkts_countries_tag.gender
    df_age_gender_bkts_index['year'] = age_gender_bkts_countries_tag.year
    df_age_gender_bkts_reshape = pd.concat([df_age_gender_bkts_reshape,df_age_gender_bkts_index])

In [17]:
#set the destination_language as the language type which helps to match the language in df_feature
df_age_gender_bkts_reshape['language'] = [x[:2] for x in df_age_gender_bkts_reshape.destination_language]
df_age_gender_bkts_reshape = df_age_gender_bkts_reshape.drop(['destination_language'], axis=1)

#change the gener into upper 
df_age_gender_bkts_reshape['gender'] = [x.upper() for x in df_age_gender_bkts_reshape.gender]

#group   by the 'age_bucket','gender','language'
df_age_gender_bkts_reshape = df_age_gender_bkts_reshape.groupby(['age_bucket','gender','language'],as_index=False).agg(np.sum)

#Inputing this kind of missing value with 0 (missing values in train and test)
df_age_gender_bkts_reshape = df_age_gender_bkts_reshape.fillna(0)

#merge the df_feature and df_countries_reshape by the laguage
df_feature = pd.merge(df_feature, df_age_gender_bkts_reshape,how='left', left_on=['age_bucket','gender','language'], right_on=['age_bucket','gender','language'])

In [20]:
#one hot encode
df_age = pd.get_dummies(df_feature.age_bucket, prefix='age_interval')
df_feature = df_feature.drop(['age_bucket'], axis=1)
df_feature = pd.concat((df_feature, df_age), axis=1)

### Begin preprocese the date feature

In [21]:
#engineering the date_account_created feature
date_account_created = df_feature["date_account_created"].tolist()
date_account_created = [datetime.strptime(x, '%Y/%m/%d') for x in date_account_created]
#generate the year month day threee feature
df_feature['dac_year'] = [ x.year for x in date_account_created]
df_feature['dac_month'] = [ x.month for x in date_account_created]
df_feature['dac_day'] = [ x.day for x in date_account_created]
df_feature['dac_week'] = np.array([d.isocalendar()[1] for d in date_account_created])

#engineering the timestamp_first_active feature
timestamp_first_active = df_feature["timestamp_first_active"].tolist()
timestamp_first_active = [datetime.strptime(str(x)[:6], '%Y%m') for x in timestamp_first_active]
#generate the year month day threee feature
df_feature['tfa_year'] = [ x.year for x in timestamp_first_active]
df_feature['tfa_month'] = [ x.month for x in timestamp_first_active]
df_feature['tfa_day'] = [ x.day for x in timestamp_first_active]
df_feature['tfa_week'] = np.array([d.isocalendar()[1] for d in timestamp_first_active])

#delet the date from the df_feature dataframe
df_feature = df_feature.drop(['date_account_created'], axis=1)
#delet the date from the df_feature dataframe
df_feature = df_feature.drop(['timestamp_first_active'], axis=1)

In [22]:
#timespans between dates
#(Computing absolute number of seconds of difference between dates, sign of the difference)
df_feature['dac_tfa_days'] = np.array([(date_account_created[i]-timestamp_first_active[i]).days for i in range(len(date_account_created))])
df_feature['sig_dac_tfa'] = np.array([np.sign((date_account_created[i]-timestamp_first_active[i]).total_seconds()) for i in range(len(date_account_created))])

Set season feature

In [23]:
Y = 2000 # dummy leap year to allow input X-02-29 (leap day)
seasons = [('winter', (date(Y,  1,  1),  date(Y,  3, 20))),
           ('spring', (date(Y,  3, 21),  date(Y,  6, 20))),
           ('summer', (date(Y,  6, 21),  date(Y,  9, 22))),
           ('autumn', (date(Y,  9, 23),  date(Y, 12, 20))),
           ('winter', (date(Y, 12, 21),  date(Y, 12, 31)))]

def get_season(now):
    if isinstance(now, datetime):
        now = now.date()
    now = now.replace(year=Y)
    return next(season for season, (start, end) in seasons
                if start <= now <= end)

df_feature['season_dac'] = np.array([get_season(dt) for dt in date_account_created])
df_feature['season_tfa'] = np.array([get_season(dt) for dt in timestamp_first_active])

In [24]:
#one-hot-encodeing of dac_season and tfa season
df_dac_season = pd.get_dummies(df_feature.season_dac, prefix='dac_season')
df_feature = df_feature.drop(['season_dac'], axis=1)
df_feature = pd.concat((df_feature, df_dac_season), axis=1)

df_tfa_season = pd.get_dummies(df_feature.season_tfa, prefix='tfa_season')
df_feature = df_feature.drop(['season_tfa'], axis=1)
df_feature = pd.concat((df_feature, df_tfa_season), axis=1)

In [25]:
#One-hot-encoding other features
ohe_feats = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']
for f in ohe_feats:
    df_feature_dummy = pd.get_dummies(df_feature[f], prefix=f)
    df_feature = df_feature.drop([f], axis=1)
    df_feature = pd.concat((df_feature, df_feature_dummy), axis=1)  

In [27]:
#One-hot-encoding destination_language features
df_destination_language = pd.get_dummies(df_feature.destination_language, prefix='destination_language')
df_feature = df_feature.drop(['destination_language'], axis=1)
df_feature = pd.concat((df_feature, df_destination_language), axis=1)

## Preprocese session dataset

In [29]:
#Filling nan with specific value ('NAN')
df_sessions.action = df_sessions.action.fillna('NAN')
df_sessions.action_type = df_sessions.action_type.fillna('NAN')
df_sessions.action_detail = df_sessions.action_detail.fillna('NAN')
df_sessions.device_type = df_sessions.device_type.fillna('NAN')

#Action values with low frequency are changed to 'OTHER'
act_freq = 100  #Threshold for frequency
act = dict(zip(*np.unique(df_sessions.action, return_counts=True)))
df_sessions.action = df_sessions.action.apply(lambda x: 'OTHER' if act[x] < act_freq else x)


# # count unique features and their counts
f_action = df_sessions.action.value_counts().argsort()
f_action_detail = df_sessions.action_detail.value_counts().argsort()
f_action_type = df_sessions.action_type.value_counts().argsort()
f_device_type = df_sessions.device_type.value_counts().argsort()

Session_data = []

#########Preparing Session data########
print('Working on Session data...')
#grouping session by id.compute features from all rows with the same id.


sess_id_group = df_sessions.groupby(['user_id'])

cont = 0
ln = len(sess_id_group)
for i in sess_id_group:
    if cont%10000 == 0:
        print("%s from %s" %(cont, ln))
        
    user_id = i[0]
    records = i[1]
    
    single_user_session = []
    single_user_session.append(user_id)
    
    sec_values = records.secs_elapsed.fillna(0).values   #These values are used later.

    #action times count, number of unique values, mean and std)
    count_action = [0] * len(f_action)
    for j,v in enumerate(records.action.values):
        count_action[f_action[v]] += 1
#     foo, c_action_uqique = np.unique(records.action.values, return_counts=True)
#     count_action += [len(c_action_uqique), np.mean(c_action_uqique), np.std(c_action_uqique)]
    single_user_session = single_user_session+ count_action
    
    #action_detail features
    #(how many times each value occurs, numb of unique values, mean and std)
    count_action_detail = [0] * len(f_action_detail)
    for j,v in enumerate(records.action_detail.values):
        count_action_detail[f_action_detail[v]] += 1 
#     foo, c_act_det_uqc = np.unique(gr.action_detail.values, return_counts=True)
#     c_act_detail += [len(c_act_det_uqc), np.mean(c_act_det_uqc), np.std(c_act_det_uqc)]
    single_user_session = single_user_session + count_action_detail
    
    #action_type features
    lag_action_type = [0] * len(f_action_type)
    count_action_type = [0] * len(f_action_type)
    for j,v in enumerate(records.action_type.values):
        count_action_type[f_action_type[v]] += 1  
#     foo, count_action_type_unique = np.unique(records.action_type.values, return_counts=True)
#     count_action_type += [len(count_action_type_unique), np.mean(count_action_type_unique), np.std(count_action_type_unique)]
    single_user_session = single_user_session + count_action_type + lag_action_type    
    
    
    #device_type features
    #(how many times each value occurs, numb of unique values, mean and std)
    count_device_type  = [0] * len(f_device_type)
    for j,v in enumerate(records.device_type .values):
        count_device_type[f_device_type[v]] += 1 
    count_device_type.append(len(np.unique(records.device_type.values)))
#     foo, count_device_type_uqique = np.unique(records.device_type.values, return_counts=True)
#     count_device_type += [len(count_device_type_uqique), np.mean(count_device_type_uqique), np.std(count_device_type_uqique)]        
    single_user_session = single_user_session + count_device_type    
   
    
    #Simple statistics about the secs_elapsed values.
    l_secs = [0] * 4
    if len(sec_values) > 0:  
        l_secs[0] = np.log(1 + np.sum(sec_values)) 
        l_secs[1] = np.log(1 + np.mean(sec_values)) 
        l_secs[2] = np.log(1 + np.std(sec_values))
        l_secs[3] = np.log(1 + np.median(sec_values))
    single_user_session = single_user_session + l_secs
    Session_data.append(single_user_session)  
    cont += 1
#Creating a dataframe with the computed features    
col_names = []    #name of the columns
for i in range(len(Session_data[0])-1):
    col_names.append('s_' + str(i)) 
#preparing objects    
Session_data = np.array(Session_data)
Session_data_ar = Session_data[:, 1:].astype(np.float16)
Session_data_id = Session_data[:, 0]   #The first element in obs is the id of the sample.

#creating the dataframe        
df_agg_sess = pd.DataFrame(Session_data_ar, columns=col_names)
df_agg_sess['id'] = Session_data_id
df_agg_sess.index = df_agg_sess.id

Working on Session data...
0 from 135483
10000 from 135483
20000 from 135483
30000 from 135483
40000 from 135483
50000 from 135483
60000 from 135483
70000 from 135483
80000 from 135483
90000 from 135483
100000 from 135483
110000 from 135483
120000 from 135483
130000 from 135483


In [31]:
#merge the data with session data
df_feature_new =pd.merge(df_feature, df_agg_sess, how='left')

In [34]:
#drop the id and set the null value as -2
df_feature_new = df_feature_new.drop(['id'], axis=1)
df_feature_new = df_feature_new.fillna(-2)  #Missing features for samples without sesssion data.
#All types of null 
#df_feature_new['all_null'] = np.array([sum(r<0) for r in df_feature_new.values])

## ！！！Output the preproseeing data as an csv file which use to train different model and do the comparing

In [ ]:
df_feature_new.to_csv(global_path+'/data/inputdata.csv',index=False)

The inner structure of the data is show in below

In [44]:
df_feature_new

,age,number_null,destination_km2_AU,destination_km2_CA,destination_km2_DE,destination_km2_ES,destination_km2_FR,destination_km2_GB,destination_km2_IT,destination_km2_NL,...,s_419,s_420,s_421,s_422,s_423,s_424,s_425,s_426,s_427,all_null
0,-1.0,2,7741220.0,9984670.0,0.0,0.0,0.0,243610.0,0.0,0.0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.000000,-2.000000,-2.000000,-2.000000,444
1,38.0,0,7741220.0,9984670.0,0.0,0.0,0.0,243610.0,0.0,0.0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.000000,-2.000000,-2.000000,-2.000000,432
2,56.0,0,7741220.0,9984670.0,0.0,0.0,0.0,243610.0,0.0,0.0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.000000,-2.000000,-2.000000,-2.000000,432
3,42.0,0,7741220.0,9984670.0,0.0,0.0,0.0,243610.0,0.0,0.0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.000000,-2.000000,-2.000000,-2.000000,432
4,41.0,1,7741220.0,9984670.0,0.0,0.0,0.0,243610.0,0.0,0.0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.000000,-2.000000,-2.000000,-2.000000,443
5,-1.0,2,7741220.0,9984670.0,0.0,0.0,0.0,243610.0,0.0,0.0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.000000,-2.000000,-2.000000,-2.000000,444
6,46.0,0,7741220.0,9984670.0,0.0,0.0,0.0,243610.0,0.0,0.0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.000000,-2.000000,-2.000000,-2.000000,432
7,47.0,0,7741220.0,9984670.0,0.0,0.0,0.0,243610.0,0.0,0.0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.000000,-2.000000,-2.000000,-2.000000,432
8,50.0,0,7741220.0,9984670.0,0.0,0.0,0.0,243610.0,0.0,0.0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.000000,-2.000000,-2.000000,-2.000000,432
9,46.0,1,7741220.0,9984670.0,0.0,0.0,0.0,243610.0,0.0,0.0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.000000,-2.000000,-2.000000,-2.000000,443
